# devlog 2024-09-24

_Author: Tyler Coles_

We recently added a framework for ADRIOs to report network activity during the fetching of data.

While we don't have an ADRIO that uses this yet, it's easy enough to mock-up a simulated download just to see what the messaging looks like. (Of course you'll want to run it yourself to get the full effect.)

In [1]:
from math import floor
from time import sleep
from epymorph import *
from epymorph.adrio.adrio import Adrio
from epymorph.data.ipm.sirs import Sirs
from epymorph.data.mm.no import No
from epymorph.event import DownloadActivity
from epymorph.geography.us_census import CountyScope
import numpy as np


class SimulateDownloadN(Adrio[np.int64]):
    """Let's pretend like we're downloading something!"""

    def evaluate_adrio(self):
        N = self.dim.nodes  # number of nodes in scope
        S = 1_000_000  # file size: 1 MB per node

        # pre-computing all of the wait times is easier
        waits = self.rng.lognormal(mean=1, sigma=0.2, size=N) / 5

        # sleep for a bit and then report some progress
        for i in range(N):
            sleep(waits[i])
            download = DownloadActivity(
                total=N * S,
                downloaded=i * S,
                download_speed=floor(S / waits[i]),
            )
            self.progress((i + 1) / N, download)

        # but our actual return value is just N fixed numbers
        return np.full(shape=N, fill_value=100_000)


rume = SingleStrataRume.build(
    ipm=Sirs(),
    mm=No(),
    init=init.SingleLocation(location=0, seed_size=100),
    scope=CountyScope.in_states_by_code(["AZ", "NM"], year=2015),
    time_frame=TimeFrame.year(2015),
    params={
        "ipm::beta": 0.4,
        "ipm::gamma": 1 / 5,
        "ipm::xi": 1 / 90,
        "mm::phi": 40.0,
        "population": SimulateDownloadN(),
    },
)

with sim_messaging():
    BasicSimulator(rume).run()

Loading __main__.SimulateDownloadN:
  |####################| 100%  (26.512s)                 
Running simulation (BasicSimulator):
• 2015-01-01 to 2016-01-01 (365 days)
• 48 geo nodes
  |####################| 100% 
Runtime: 1.608s
